In [12]:
import torch
import torchaudio
import sys
sys.path.append("/fs/scratch/pi_cr_soundsee/archive/workspace/hul2pi/Uni-ControlNet-Cross-Attention")
from deepafx_st.system import System
from deepafx_st.utils import DSPMode

In [16]:
ckpt = '/fs/scratch/pi_cr_soundsee/archive/workspace/hul2pi/Uni-ControlNet-Cross-Attention/checkpoints/style/libritts/autodiff/lightning_logs/version_1/checkpoints/epoch=367-step=1226911-val-libritts-autodiff.ckpt'

use_dsp = False
system = System.load_from_checkpoint(ckpt, dsp_mode=DSPMode.NONE, batch_size=1).eval()
system.to("cuda")

System(
  (processor): AutodiffChannel(
    (peq): ParametricEQ()
    (comp): Compressor()
  )
  (encoder): SpectralEncoder(
    (encoder): EfficientNet(
      (_conv_stem): Conv2dStaticSamePadding(
        1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        (static_padding): ZeroPad2d((1, 1, 0, 1))
      )
      (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_blocks): ModuleList(
        (0): MBConvBlock(
          (_depthwise_conv): Conv2dStaticSamePadding(
            32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
            (static_padding): ZeroPad2d((1, 1, 1, 1))
          )
          (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (_se_reduce): Conv2dStaticSamePadding(
            32, 8, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): Identity()
          )
          (_se_expand): Conv2dStaticSamePadding(


In [17]:
# load audio data
x, x_sr = torchaudio.load('/fs/scratch/pi_cr_soundsee/archive/workspace/hul2pi/Uni-ControlNet-Cross-Attention/examples/voice_raw.wav')
r, r_sr = torchaudio.load('/fs/scratch/pi_cr_soundsee/archive/workspace/hul2pi/Uni-ControlNet-Cross-Attention/examples/voice_produced.wav')

In [18]:
# resample if needed
if x_sr != 24000:
    print("Resampling to 24000 Hz...")
    x_24000 = torch.tensor(resampy.resample(x.view(-1).numpy(), x_sr, 24000))
    x_24000 = x_24000.view(1, -1)
else:
    x_24000 = x

if r_sr != 24000:
    print("Resampling to 24000 Hz...")
    r_24000 = torch.tensor(resampy.resample(r.view(-1).numpy(), r_sr, 24000))
    r_24000 = r_24000.view(1, -1)
else:
    r_24000 = r

# peak normalize to -12 dBFS
x_24000 = x_24000[0:1, : 24000 * 5]
x_24000 /= x_24000.abs().max()
x_24000 *= 10 ** (-12 / 20.0)
x_24000 = x_24000.view(1, 1, -1)

# peak normalize to -12 dBFS
r_24000 = r_24000[0:1, : 24000 * 5]
r_24000 /= r_24000.abs().max()
r_24000 *= 10 ** (-12 / 20.0)
r_24000 = r_24000.view(1, 1, -1)

x_24000 = x_24000.to("cuda")
r_24000 = r_24000.to("cuda")

In [19]:
with torch.no_grad():
    y_hat, p, e = system(x_24000, r_24000)

In [20]:
print(e)

tensor([[-0.0108, -0.0019,  0.0390,  ..., -0.0178,  0.0033, -0.0192]],
       device='cuda:0')
